In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

/home/bastin/envs/ml_ppl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'
train_data = TabularDataset(f'{data_url}train.csv')
train_data.head()

,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [3]:
label = 'signature'
train_data[label].describe()

count    10000.000000
mean        -0.022000
std          3.025166
min        -12.000000
25%         -2.000000
50%          0.000000
75%          2.000000
max         12.000000
Name: signature, dtype: float64

In [4]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240628_043148"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jan 11 04:09:03 UTC 2024
CPU Count:          16
Memory Avail:       5.69 GB / 7.68 GB (74.1%)
Disk Space Avail:   923.86 GB / 1006.85 GB (91.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training tim

In [5]:
test_data = TabularDataset(f'{data_url}test.csv')

Loaded data from: https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/test.csv | Columns = 19 / 19 | Rows = 5000 -> 5000


In [6]:
y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

0   -4
1   -2
2    0
3    4
4    2
Name: signature, dtype: int64

In [7]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9502,
 'balanced_accuracy': 0.7556460568397135,
 'mcc': 0.9389682132276683}

In [8]:
leader_board = predictor.leaderboard(test_data)

In [9]:
import pandas as pd

In [10]:
ldb_df = pd.DataFrame(leader_board)
ldb_df.head()

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.9502,0.962963,accuracy,0.562376,0.227299,17.142752,0.007840,0.000624,0.158975,2,True,14
1,LightGBM,0.9456,0.955956,accuracy,0.090974,0.016275,2.312691,0.090974,0.016275,2.312691,1,True,5
2,XGBoost,0.9448,0.956957,accuracy,0.262545,0.031092,4.748956,0.262545,0.031092,4.748956,1,True,11
3,LightGBMLarge,0.9444,0.949950,accuracy,0.241745,0.038807,5.444538,0.241745,0.038807,5.444538,1,True,13
4,CatBoost,0.9432,0.955956,accuracy,0.016479,0.003866,19.684905,0.016479,0.003866,19.684905,1,True,8


In [11]:
ldb_df.columns

Index(['model', 'score_test', 'score_val', 'eval_metric', 'pred_time_test',
       'pred_time_val', 'fit_time', 'pred_time_test_marginal',
       'pred_time_val_marginal', 'fit_time_marginal', 'stack_level',
       'can_infer', 'fit_order'],
      dtype='object')

In [12]:
ldb_df.to_csv('modelLeaderBoardAutoMLGloun.csv')